In [13]:
import numpy as np
m = 3
num_classes = 80
input_shape = np.array([416,416])
num_layers = 3
anchors = np.array([[10,13],
                    [16,30],
                    [33,23],
                    [30,61],
                    [62,45],
                    [59,119],
                    [116,90],
                    [156,198],
                    [373,326]])
anchor_mask = [[6,7,8], [3,4,5], [0,1,2]]
grid_shapes = [input_shape//{0:32, 1:16, 2:8}[l] for l in range(num_layers)]
y_true = [np.zeros((m,grid_shapes[l][0],grid_shapes[l][1],len(anchor_mask[l]),5+num_classes),
        dtype='float32') for l in range(num_layers)]
print(y_true[1].shape)
anchors = np.expand_dims(anchors, 0)
anchor_maxes = anchors / 2.
anchor_mins = -anchor_maxes
print(anchor_maxes.shape)

(3, 26, 26, 3, 85)
(1, 9, 2)


In [20]:
# already scale to adap with 416x416
box1 = np.array([[50,100,150,200,0]])
box2 = np.array([[30,50,200,120,3]])
box3 = np.array([[20,10,100,80,2]])
box = np.concatenate([box1,box2,box3], axis = 0)
box = np.expand_dims(box, 0)
true_boxes = np.concatenate([box,box,box], axis = 0)
true_boxes = np.array(true_boxes, dtype='float32')

boxes_xy = (true_boxes[..., 0:2] + true_boxes[..., 2:4]) // 2
boxes_wh = true_boxes[..., 2:4] - true_boxes[..., 0:2]
true_boxes[..., 0:2] = boxes_xy/input_shape[::-1]
true_boxes[..., 2:4] = boxes_wh/input_shape[::-1]

valid_mask = boxes_wh[..., 0]>0

wh = boxes_wh[1, valid_mask[1]]
print(wh.shape)
wh = np.expand_dims(wh, -2)
# print(wh.shape)
# print(anchors.shape)
box_maxes = wh / 2.
box_mins = -box_maxes

intersect_mins = np.maximum(box_mins, anchor_mins)
# print(intersect_mins.shape)
intersect_maxes = np.minimum(box_maxes, anchor_maxes)
intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)


intersect_area = intersect_wh[..., 0] * intersect_wh[..., 1]
# print(intersect_area.shape)
box_area = wh[..., 0] * wh[..., 1]
# print(box_area)
anchor_area = anchors[..., 0] * anchors[..., 1]
# print(anchor_area)
iou = intersect_area / (box_area + anchor_area - intersect_area)
temp = (box_area/anchor_area)
# print(temp)
# print(intersect_area)
# print(iou)
print(temp==iou)
# Find best anchor for each true box
best_anchor = np.argmax(iou, axis=-1)
print(best_anchor)

(3, 2)
[[False False False False False False False  True  True]
 [False False False False False False False False  True]
 [False False False False False False  True  True  True]]
[6 6 6]


In [26]:
b = 0
for t, n in enumerate(best_anchor):
    for l in range(num_layers):
        if n in anchor_mask[l]:
            i = np.floor(true_boxes[b,t,0]*grid_shapes[l][1]).astype('int32')
            j = np.floor(true_boxes[b,t,1]*grid_shapes[l][0]).astype('int32')
            k = anchor_mask[l].index(n)
            c = true_boxes[b,t, 4].astype('int32')
            y_true[l][b, j, i, k, 0:4] = true_boxes[b,t, 0:4]
            y_true[l][b, j, i, k, 4] = 1
            y_true[l][b, j, i, k, 5+c] = 1

In [16]:
y_true[0].shape

(3, 13, 13, 3, 85)

In [25]:
true_boxes.shape

(3, 3, 5)